<a href="https://colab.research.google.com/github/chegmarco1989/AdminLTE/blob/master/demo_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

class Layer:
    def __init__(self, input_size, output_size, activation=torch.relu):
        self.weights = torch.nn.Parameter(torch.randn(output_size, input_size) * 0.1)
        self.biases = torch.zeros(output_size, requires_grad=True)
        self.activation = activation

    def forward(self, x):
        """
        x : Tensor de taille (batch_size, input_size)
        Retourne : Tensor de taille (batch_size, output_size)
        """
        z = x @ self.weights.T + self.biases  # poids : (output, input) → transposé pour (input, output)
        return self.activation(z)

    def parameters(self):
        return [self.weights, self.biases]



In [ ]:
class NeuralNetwork:
    def __init__(self, layer_sizes, activation=torch.relu):
        """
        layer_sizes : liste comme [2, 4, 1] pour 2 entrées, 1 couche cachée de 4 neurones, 1 sortie
        activation : fonction d'activation pour toutes les couches sauf la dernière
        """
        self.layers = []
        for i in range(len(layer_sizes) - 1):
            act = activation if i < len(layer_sizes) - 2 else lambda x: x
            self.layers.append(Layer(layer_sizes[i], layer_sizes[i+1], activation=act))

    def forward(self, x):
        """
        x : Tensor de taille (input_size,)
        """
        for layer in self.layers:
            x = layer.forward(x)
        return x

    def parameters(self):
        """
        Retourne tous les poids et biais du réseau sous forme de liste
        """
        params = []
        for layer in self.layers:
            params.extend(layer.parameters())
        return params


In [ ]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.view(-1))
])

# Chargement du dataset
train_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)

# Prendre un petit batch pour la démo (par ex. 32 images)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

# Récupère un batch d'images
for x_batch, y_batch in train_loader:
    print("Shape des entrées :", x_batch.shape)  # (32, 784)
    print("Shape des sorties :", y_batch.shape)  # (32,)
    break


100%|██████████| 9.91M/9.91M [00:00<00:00, 51.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 2.05MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.4MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.72MB/s]


Shape des entrées : torch.Size([32, 784])
Shape des sorties : torch.Size([32])


In [ ]:
def to_one_hot(labels, num_classes=10):
    """
    labels : Tensor de taille (batch_size,)
    Retourne : Tensor de taille (batch_size, num_classes)
    """
    return torch.eye(num_classes)[labels]

In [ ]:
net = NeuralNetwork([784, 16, 16, 10])

def compute_accuracy(predictions, labels):
    predicted_classes = predictions.argmax(dim=1)
    correct = (predicted_classes == labels).sum().item()
    total = labels.size(0)
    return correct / total


total_correct = 0
total_samples = 0

for x_batch, y_batch in test_loader:
    outputs = net.forward(x_batch)
    preds = outputs.argmax(dim=1)
    total_correct += (preds == y_batch).sum().item()
    total_samples += y_batch.size(0)

print(f"Accuracy totale : {100 * total_correct / total_samples:.2f}%")




Accuracy totale : 11.26%


In [ ]:
def train(model, train_loader, test_loader, lr=0.1, epochs=5):
    for epoch in range(epochs):
        total_loss = 0.0
        total_acc = 0.0
        batch_count = 0

        for x_batch, y_batch in train_loader:
            # Forward
            outputs = model.forward(x_batch)
            targets = to_one_hot(y_batch, num_classes=10)

            # Loss
            loss = ((outputs - targets) ** 2).mean()


            # Backward
            loss.backward()


            # Update params
            with torch.no_grad():
                for param in model.parameters():
                  if param.grad is not None:
                      param -= lr * param.grad
                      param.grad.zero_()


            # Track metrics
            total_loss += loss.item()
            total_acc += compute_accuracy(outputs, y_batch)
            batch_count += 1

        avg_loss = total_loss / batch_count
        avg_acc = total_acc / batch_count

        # Évaluation sur test set
        test_acc = 0.0
        test_batches = 0
        with torch.no_grad():
            for x_test, y_test in test_loader:
                preds = model.forward(x_test)
                test_acc += compute_accuracy(preds, y_test)
                test_batches += 1

        test_acc /= test_batches

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {avg_loss:.4f} | Train Acc: {avg_acc*100:.2f}% | Test Acc: {test_acc*100:.2f}%")

In [ ]:
net = NeuralNetwork([784, 16, 16, 10])

train(net, train_loader, test_loader, lr=0.1, epochs=10)


Epoch 1/10 | Train Loss: 0.0675 | Train Acc: 52.23% | Test Acc: 77.52%
Epoch 2/10 | Train Loss: 0.0386 | Train Acc: 82.33% | Test Acc: 86.82%
Epoch 3/10 | Train Loss: 0.0278 | Train Acc: 87.18% | Test Acc: 88.93%
Epoch 4/10 | Train Loss: 0.0233 | Train Acc: 88.91% | Test Acc: 90.13%
Epoch 5/10 | Train Loss: 0.0210 | Train Acc: 89.83% | Test Acc: 90.88%
Epoch 6/10 | Train Loss: 0.0194 | Train Acc: 90.52% | Test Acc: 91.19%
Epoch 7/10 | Train Loss: 0.0181 | Train Acc: 91.00% | Test Acc: 91.50%
Epoch 8/10 | Train Loss: 0.0171 | Train Acc: 91.45% | Test Acc: 91.94%
Epoch 9/10 | Train Loss: 0.0163 | Train Acc: 91.78% | Test Acc: 92.19%
Epoch 10/10 | Train Loss: 0.0156 | Train Acc: 92.14% | Test Acc: 92.22%


In [ ]:
import torch.nn.functional as F

class ConvLayer:
    def __init__(self, in_channels, out_channels, kernel_size, activation=torch.relu):
        self.weights = torch.nn.Parameter(torch.randn(out_channels, in_channels, kernel_size, kernel_size) * 0.1)
        self.biases = torch.nn.Parameter(torch.zeros(out_channels))
        self.activation = activation

    def forward(self, x):
        # x : (batch, in_channels, height, width)
        z = F.conv2d(x, self.weights, self.biases, padding=1)
        return self.activation(z)

    def parameters(self):
        return [self.weights, self.biases]

class MaxPool2d:
    def __init__(self, kernel_size=2, stride=2):
        self.kernel_size = kernel_size
        self.stride = stride

    def forward(self, x):
        return F.max_pool2d(x, self.kernel_size, self.stride)

    def parameters(self):
        return []

class ReshapeLayer:
    def __init__(self, shape):
        """
        shape : tuple, la nouvelle forme **sans** la dimension batch,
                par ex. (1, 28, 28)
        """
        self.shape = shape

    def forward(self, x):
        # x : (batch, 784)
        return x.view(x.size(0), *self.shape)

    def parameters(self):
        return []



In [ ]:
class FlexibleNeuralNetwork:
    def __init__(self, layers):
        """
        layers : liste d'objets, chacun doit avoir .forward(x) et .parameters()
                 (ex : ConvLayer, Layer, torch.nn.Flatten)
        """
        self.layers = layers

    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x) if hasattr(layer, 'forward') else layer(x)
        return x

    def parameters(self):
        params = []
        for layer in self.layers:
            if hasattr(layer, 'parameters'):
                params.extend(layer.parameters())
        return params


In [ ]:
net = FlexibleNeuralNetwork([
    ReshapeLayer((1,28,28)),
    ConvLayer(1, 4, 3),
    MaxPool2d(),
    ConvLayer(4, 8, 3),
    MaxPool2d(),
    torch.nn.Flatten(),
    Layer(392, 16),
    Layer(16,16),
    Layer(16,10)
])



train(net,train_loader,test_loader,lr = 0.1, epochs=10)

Epoch 1/10 | Train Loss: 0.0792 | Train Acc: 35.20% | Test Acc: 77.07%
Epoch 2/10 | Train Loss: 0.0283 | Train Acc: 85.16% | Test Acc: 90.32%
Epoch 3/10 | Train Loss: 0.0182 | Train Acc: 91.05% | Test Acc: 92.57%
Epoch 4/10 | Train Loss: 0.0145 | Train Acc: 93.10% | Test Acc: 94.42%
Epoch 5/10 | Train Loss: 0.0123 | Train Acc: 94.14% | Test Acc: 95.02%
Epoch 6/10 | Train Loss: 0.0109 | Train Acc: 94.94% | Test Acc: 95.44%
Epoch 7/10 | Train Loss: 0.0099 | Train Acc: 95.40% | Test Acc: 96.05%
Epoch 8/10 | Train Loss: 0.0091 | Train Acc: 95.86% | Test Acc: 96.09%
Epoch 9/10 | Train Loss: 0.0085 | Train Acc: 96.15% | Test Acc: 96.27%
Epoch 10/10 | Train Loss: 0.0080 | Train Acc: 96.34% | Test Acc: 96.23%
